In [1]:
#import stuffz
import nltk
import numpy as np
import pandas as pd

In [2]:
from nltk.metrics import edit_distance

In [3]:
import re

In [4]:
crosscheck_df = pd.read_csv('cross_check_test.csv')

In [5]:
crosscheck_df.head()

,id,status,record_id,source_type,source_gender,source_title,source_date,source_publisher,source_pub_viaf,source_genre,...,target_type,target_gender,target_title,target_date,target_publisher,target_pub_viaf,target_genre,target_price,target_author_viaf,target_author_name
0,152,done,136129795,single_focus,m,Times Before the Reformation,1880.0,Robert Carter & Brothers,NaN,historical nonfiction,...,single_focus,m,"Times before the Reformation, with an account ...",1880.0,New York: Robert Carter & Brothers,NaN,Nonfiction,NaN,http://viaf.org/viaf/93207229,William Dinwiddie
1,662,done,124766315,single_focus,f,One Man Who Was Content,1897.0,The Century Company,NaN,fiction,...,single_focus,f,One Man Who Was Content,1897.0,New York: The Century Company,NaN,Fiction,$1.00,http://viaf.org/viaf/44455381,Schuyler Van Rensselaer
2,236,done,124905465,multi,NaN,NaN,NaN,NaN,NaN,NaN,...,multi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,660,done,136243777,multi,multi,NaN,NaN,NaN,NaN,NaN,...,cluster,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,250,done,136680305,not_review,NaN,NaN,NaN,NaN,NaN,NaN,...,not_review,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
pd.Timestamp(1880)

Timestamp('1970-01-01 00:00:00.000001880')

### Goals:
+ Calculate interrater reliability
+ Type: single_focus vs. not single_focus
+ Title, author, & publisher: levenshtein distance, disregard punc and capitalization
+ VIAF: just the number

will add other stuff as i remember

In [6]:
crosscheck_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 23 columns):
id                    35 non-null int64
status                35 non-null object
record_id             35 non-null int64
source_type           35 non-null object
source_gender         18 non-null object
source_title          17 non-null object
source_date           17 non-null float64
source_publisher      17 non-null object
source_pub_viaf       0 non-null float64
source_genre          17 non-null object
source_price          10 non-null object
source_author_viaf    15 non-null object
source_author_name    17 non-null object
target_type           35 non-null object
target_gender         17 non-null object
target_title          17 non-null object
target_date           17 non-null float64
target_publisher      17 non-null object
target_pub_viaf       0 non-null float64
target_genre          17 non-null object
target_price          10 non-null object
target_author_viaf    17 non-null ob

In [7]:
source_ex = crosscheck_df.source_title[0]
target_ex = crosscheck_df.target_title[0]

In [8]:
' '.join(re.split('[^a-zA-Z]', target_ex))

'Times before the Reformation  with an account of fra Girolamo Savonarolam the Friar of Florence'

In [9]:
source_ex = ' '.join([w.lower() for w in re.split('[^a-zA-Z]', source_ex) if w.isalpha()])
target_ex = ' '.join([w.lower() for w in re.split('[^a-zA-Z]', target_ex) if w.isalpha()])

In [10]:
source_ex

'times before the reformation'

In [11]:
target_ex

'times before the reformation with an account of fra girolamo savonarolam the friar of florence'

In [12]:
(source_ex in target_ex) or (target_ex in source_ex)

True

In [47]:
for x, y in enumerate(crosscheck_df.source_author_name[6].split(';')):
    print(x, y)

0 William Allen White
1  Helen Ogden Mahin


In [42]:
crosscheck_df.target_author_name[6].split(';')

['William Allen White']

In [48]:
def flagSimilarity(source, target):
    """
    Cleans punc and lowercases. If source/target are same but one includes extra text, returns a yellow flag.
    Otherwise if source/target have edit_distance > 3, returns a red flag.
    """
    source = ' '.join([w.lower() for w in re.split('[^a-zA-Z]', source) if w.isalpha()])
    target = ' '.join([w.lower() for w in re.split('[^a-zA-Z]', target) if w.isalpha()])
    if source == target:
        return 'green'
    elif (source_ex in target_ex) or (target_ex in source_ex):
        return 'yellow'
    elif edit_distance(source_ex, target_ex) > 3:
        return 'red'

In [43]:
def cleanReviewInfo(text):
    return ' '.join([w.lower() for w in re.split('[^a-zA-Z]', text) if w.isalpha()])

In [44]:
cleanReviewInfo('William Allen White')

'william allen white'

In [53]:
def compNames(source, target):
    source = source.split(';')
    target = target.split(';')
    if len(source) != len(target):
        return 'inequal_author_count'
    else:
        edit_distances = []
        source = [cleanReviewInfo(x) for x in source]
        target = [cleanReviewInfo(x) for x in target]
        for x, y in enumerate(source):
            edit_distances.append(edit_distance(source[x], target[x]))
        return edit_distances

In [14]:
flagSimilarity(crosscheck_df.source_title[0], crosscheck_df.target_title[0])

'yellow'

In [15]:
def compReviewType(source_type, target_type):
    if source_type == target_type:
        return 'green'
    elif ((source_type == 'not_review' and target_type != 'not_review') or 
          (target_type == 'not_review' and source_type != 'not_review')):
        return 'red'
    elif ((source_type == 'single_focus' and target_type != 'single_focus') or
          (target_type == 'single_focus' and source_type != 'single_focus')):
        return 'red'
    else: return 'green'

In [ ]:
def relabelReviewType(type):
    if type == 'single_focus':
        return 
    if type == 'not_review'
        return type
    else:
        return 'not_single'

In [16]:
compReviewType(crosscheck_df.source_type[0], crosscheck_df.target_type[0])

'green'

In [19]:
viaf_ex = 'http://viaf.org/viaf/93207229'

In [22]:
viaf_ex.split('/')[-1]

'93207229'

In [23]:
def compVIAF(source_VIAF, target_VIAF):
    return source_VIAF.split('/')[-1] == target_VIAF.split('/')[-1]

In [31]:
for id in crosscheck_df.index:
    try:
        print(id, compVIAF(crosscheck_df.source_author_viaf[id], crosscheck_df.target_author_viaf[id]))
    except:
        print(id, 'N/A')

0 N/A
1 True
2 N/A
3 N/A
4 N/A
5 N/A
6 False
7 N/A
8 N/A
9 N/A
10 False
11 True
12 N/A
13 N/A
14 True
15 False
16 True
17 N/A
18 N/A
19 N/A
20 True
21 N/A
22 True
23 N/A
24 True
25 N/A
26 True
27 True
28 True
29 True
30 True
31 N/A
32 N/A
33 N/A
34 N/A


In [38]:
for id in crosscheck_df.index:
    try:
        color = flagSimilarity(crosscheck_df.source_title[id], crosscheck_df.target_title[id])
        if ((color == 'yellow') or (color == 'red')):
            print(id, color, '\n', crosscheck_df.source_title[id], '\n', crosscheck_df.target_title[id])
            print()
    except:
        pass

0 yellow 
 Times Before the Reformation 
 Times before the Reformation, with an account of fra Girolamo Savonarolam the Friar of Florence

11 yellow 
 Légendes Prophéties et Superstitions de la Guerre 
 Legendes, Propheties, et Superstitions de la Guerre

24 yellow 
 The Goede Vrouw of Mana-ha-ta, at Home and in Society 
 The Goede Vrouw of Manu-ha-ta At Home and in Society

28 yellow 
 Missouri 
 Missouri: A Bone of Contention

29 yellow 
 The Goede Vrouw of Mana-ha-ta, at Home and in Society 
 The Goede Vrouw of Manu-ha-ta At Home and in Society



In [61]:
for id in crosscheck_df.index:
    try:
        status = compNames(crosscheck_df.source_author_name[id], crosscheck_df.target_author_name[id])
        if status == 'inequal_author_count':
            print(id, status)
            print()
        if sum(status) == 0:
            pass
        else:
            for x, y in enumerate(status):
                print(id, '\n', crosscheck_df.source_author_name[id], '\n', crosscheck_df.target_author_name[id])
                print(' edit distance:', y)
                print()
    except:
        pass

0 
 William Didwiddie 
 William Dinwiddie
 edit distance: 1

6 inequal_author_count

10 
 J. Estlin Carpenter 
 William B Carpenter
 edit distance: 9

15 inequal_author_count

16 
 Lord Charles Beresford, M.P. 
 Lord Charles Beresford
 edit distance: 4

20 
 Madame Mary DuClaux 
 Mary DuClaux
 edit distance: 7

24 
 Mrs. John King Van Rensselaer 
 John King Van Rensselaer
 edit distance: 4

29 
 Mrs. John King Van Rensselaer 
 John King Van Rensselaer
 edit distance: 4

30 
 Arthur Twinning Hadley 
 Arthur Twining Hadley
 edit distance: 1



#### Notes:
Should remove titles before analysis - the case of Mrs. John King Van Rensselaer is difficult though